In [22]:
import pandas as pd
import glob

## Clean Packet Capture

In [60]:
# List of all csv in a dir
csv_files = glob.glob('captures1_v2/clean/*.csv')

clean_df = pd.DataFrame()

# append to combined dataframe
for csv_file in csv_files:
    df = pd.read_csv(csv_file)
    capture = csv_file.split('/')[-1].replace('.csv', '')
    df['CaptureName'] = capture
    clean_df = pd.concat([clean_df, df])

clean_df['AttackName'] = 'Clean'

clean_df.head()


,No.,Time,SrcIP,DstIP,Protocol,Length,Info,SrcMAC,SrcMACResolved,SrcOUIResolved,...,DstPort,DstMACResolved.1,SYNFlag,ACKFlag,ProtocolType,EpochTime,RelativeTime,TimeDelta,CaptureName,AttackName
0,1,0.000000,172.27.224.70,172.27.224.250,TCP,60,49499 > 502 [ACK] Seq=1 Ack=1 Win=65051 Len=0,00:0c:29:9d:9e:9e,VMware_9d:9e:9e,"VMware, Inc.",...,502.0,Telemech_09:51:3b,Not set,Set,TCP,1.535046e+09,0.000000,0.000000,"clean\eth2dump-clean-0,5h_1",Clean
1,2,0.020940,HewlettP_8e:40:b3,Spanning-tree-(for-bridges)_00,STP,64,RST. Root = 32768/0/00:18:6e:d7:8a:c0 Cost = ...,d0:7e:28:8e:40:b3,HewlettP_8e:40:b3,Hewlett Packard,...,NaN,Spanning-tree-(for-bridges)_00,NaN,NaN,STP,1.535046e+09,0.020940,0.020940,"clean\eth2dump-clean-0,5h_1",Clean
2,3,0.094309,172.27.224.70,172.27.224.250,Modbus/TCP,66,"Query: Trans: 0; Unit: 1, Func: 3: ...",00:0c:29:9d:9e:9e,VMware_9d:9e:9e,"VMware, Inc.",...,502.0,Telemech_09:51:3b,Not set,Set,Modbus/TCP,1.535046e+09,0.094309,0.073369,"clean\eth2dump-clean-0,5h_1",Clean
3,4,0.097427,172.27.224.250,172.27.224.70,Modbus/TCP,85,"Response: Trans: 0; Unit: 1, Func: 3: ...",00:80:f4:09:51:3b,Telemech_09:51:3b,Telemechanique Electrique,...,49499.0,VMware_9d:9e:9e,Not set,Set,Modbus/TCP,1.535046e+09,0.097427,0.003118,"clean\eth2dump-clean-0,5h_1",Clean
4,5,0.311972,172.27.224.70,172.27.224.250,TCP,60,49499 > 502 [ACK] Seq=13 Ack=32 Win=65020 Len=0,00:0c:29:9d:9e:9e,VMware_9d:9e:9e,"VMware, Inc.",...,502.0,Telemech_09:51:3b,Not set,Set,TCP,1.535046e+09,0.311972,0.214545,"clean\eth2dump-clean-0,5h_1",Clean


In [24]:
# Check for missing values
missing_values = clean_df.isnull().sum()
print(missing_values)

No.                     0
Time                    0
SrcIP                   0
DstIP                   0
Protocol                0
Length                  0
Info                    0
SrcMAC                  0
SrcMACResolved          0
SrcOUIResolved          0
SrcPort             40771
SequenceNumber      46224
SrcOUI                  0
DstMAC                  0
DstMACResolved          0
DstOUI                  0
DstOUResolved       35895
DstPort             40771
DstMACResolved.1        0
SYNFlag             46224
ACKFlag             46224
ProtocolType            0
EpochTime               0
RelativeTime            0
TimeDelta               0
AttackName              0
dtype: int64


In [25]:
# Count the number of packets where SrcPort is not applicable (null)
srcport_not_applicable = clean_df['SrcPort'].isnull().sum()
print(f"Number of packets where SrcPort is not applicable (null): {srcport_not_applicable}")

# Count the number of packets where DstPort is not applicable (null)
dstport_not_applicable = clean_df['DstPort'].isnull().sum()
print(f"Number of packets where DstPort is not applicable (null): {dstport_not_applicable}")

# Count the number of packets where SrcPort should be applicable but is null
srcport_missing = clean_df[(clean_df['Protocol'] == 'TCP') | (clean_df['Protocol'] == 'UDP')]['SrcPort'].isnull().sum()
print(f"Number of packets where SrcPort should be applicable but is null: {srcport_missing}")

# Count the number of packets where DstPort should be applicable but is null
dstport_missing = clean_df[(clean_df['Protocol'] == 'TCP') | (clean_df['Protocol'] == 'UDP')]['DstPort'].isnull().sum()
print(f"Number of packets where DstPort should be applicable but is null: {dstport_missing}")


Number of packets where SrcPort is not applicable (null): 40771
Number of packets where DstPort is not applicable (null): 40771
Number of packets where SrcPort should be applicable but is null: 0
Number of packets where DstPort should be applicable but is null: 0


In [26]:
# Count the number of packets where SequenceNumber is not applicable (null)
seqnum_not_applicable = clean_df['SequenceNumber'].isnull().sum()
print(f"Number of packets where SequenceNumber is not applicable (null): {seqnum_not_applicable}")

# Count the number of packets where SequenceNumber should be applicable but is null
seqnum_missing = clean_df[(clean_df['Protocol'] == 'TCP')]['SequenceNumber'].isnull().sum()
print(f"Number of packets where SequenceNumber should be applicable but is null: {seqnum_missing}")


Number of packets where SequenceNumber is not applicable (null): 46224
Number of packets where SequenceNumber should be applicable but is null: 0


In [27]:
# Count the number of packets where SYNFlag is not applicable (null)
synflag_not_applicable = clean_df['SYNFlag'].isnull().sum()
print(f"Number of packets where SYNFlag is not applicable (null): {synflag_not_applicable}")

# Count the number of packets where ACKFlag is not applicable (null)
ackflag_not_applicable = clean_df['ACKFlag'].isnull().sum()
print(f"Number of packets where ACKFlag is not applicable (null): {ackflag_not_applicable}")

# Count the number of packets where SYNFlag should be applicable but is null
synflag_missing = clean_df[(clean_df['Protocol'] == 'TCP')]['SYNFlag'].isnull().sum()
print(f"Number of packets where SYNFlag should be applicable but is null: {synflag_missing}")

# Count the number of packets where ACKFlag should be applicable but is null
ackflag_missing = clean_df[(clean_df['Protocol'] == 'TCP')]['ACKFlag'].isnull().sum()
print(f"Number of packets where ACKFlag should be applicable but is null: {ackflag_missing}")


Number of packets where SYNFlag is not applicable (null): 46224
Number of packets where ACKFlag is not applicable (null): 46224
Number of packets where SYNFlag should be applicable but is null: 0
Number of packets where ACKFlag should be applicable but is null: 0


In [28]:
# Count the number of packets where DstOUResolved is not applicable (null)
dstou_not_applicable = clean_df['DstOUResolved'].isnull().sum()
print(f"Number of packets where DstOUResolved is not applicable (null): {dstou_not_applicable}")

# Count the number of packets where DstOUResolved should be applicable but is null
dstou_missing = clean_df['DstOUResolved'].isnull().sum()  # No specific protocol condition since DstOUResolved can be applicable to various protocols
print(f"Number of packets where DstOUResolved should be applicable but is null: {dstou_missing}")


Number of packets where DstOUResolved is not applicable (null): 35895
Number of packets where DstOUResolved should be applicable but is null: 35895


In [29]:
# Count the number of packets where DstOUResolved is not applicable (null)
dstou_not_applicable = clean_df['DstOUResolved'].isnull().sum()
print(f"Number of packets where DstOUResolved is not applicable (null): {dstou_not_applicable}")

# Count the number of packets where DstOUResolved should be applicable but is null
dstou_missing = clean_df[((clean_df['Protocol'] == 'TCP') | (clean_df['Protocol'] == 'UDP')) & clean_df['DstOUResolved'].isnull()].shape[0]
print(f"Number of packets where DstOUResolved should be applicable but is null: {dstou_missing}")


Number of packets where DstOUResolved is not applicable (null): 35895
Number of packets where DstOUResolved should be applicable but is null: 1286


In [ ]:

# 1. Handling missing values in SrcPort and DstPort
clean_df['SrcPort'].fillna(-1, inplace=True)  # Filling missing SrcPort with -1
clean_df['DstPort'].fillna(-1, inplace=True)  # Filling missing DstPort with -1

# 2. Handling missing values in SequenceNumber
# Assuming TCP packets have SequenceNumber and using median imputation
median_seqnum = clean_df[clean_df['Protocol'] == 'TCP']['SequenceNumber'].median()
clean_df['SequenceNumber'].fillna(median_seqnum, inplace=True)

# 3. Handling missing values in DstOUResolved
clean_df['DstOUResolved'].fillna('Unknown', inplace=True)  # Filling missing DstOUResolved with 'Unknown'

# 4. Handling missing values in SYNFlag and ACKFlag
clean_df['SYNFlag'].fillna(False, inplace=True)  # Filling missing SYNFlag with False
clean_df['ACKFlag'].fillna(False, inplace=True)  # Filling missing ACKFlag with False

In [31]:
# Check for missing values
missing_values = clean_df.isnull().sum()
print(missing_values)

No.                 0
Time                0
SrcIP               0
DstIP               0
Protocol            0
Length              0
Info                0
SrcMAC              0
SrcMACResolved      0
SrcOUIResolved      0
SrcPort             0
SequenceNumber      0
SrcOUI              0
DstMAC              0
DstMACResolved      0
DstOUI              0
DstOUResolved       0
DstPort             0
DstMACResolved.1    0
SYNFlag             0
ACKFlag             0
ProtocolType        0
EpochTime           0
RelativeTime        0
TimeDelta           0
AttackName          0
dtype: int64


In [32]:
# Convert EpochTime to datetime format and create 'date_time' column
clean_df['date_time'] = pd.to_datetime(df['EpochTime'], unit='s')

## Man in the Middle Attack

In [33]:
csv_files = glob.glob('captures1_v2/mitm/*.csv')

mitm_df = pd.DataFrame()

for csv_file in csv_files:
    df = pd.read_csv(csv_file)
    mitm_df = pd.concat([mitm_df, df])

mitm_df['AttackName'] = 'mitm'

mitm_df.head()

,No.,Time,SrcIP,DstIP,Protocol,Length,Info,SrcMAC,SrcMACResolved,SrcOUIResolved,...,DstOUResolved,DstPort,DstMACResolved.1,SYNFlag,ACKFlag,ProtocolType,EpochTime,RelativeTime,TimeDelta,AttackName
0,1,0.000000,172.27.224.70,172.27.224.250,TCP,60,49499 > 502 [ACK] Seq=1 Ack=1 Win=65237 Len=0,00:0c:29:9d:9e:9e,VMware_9d:9e:9e,"VMware, Inc.",...,Telemechanique Electrique,502.0,Telemech_09:51:3b,Not set,Set,TCP,1.535061e+09,0.000000,0.000000,mitm
1,2,0.094173,172.27.224.70,172.27.224.250,Modbus/TCP,66,"Query: Trans: 0; Unit: 1, Func: 3: ...",00:0c:29:9d:9e:9e,VMware_9d:9e:9e,"VMware, Inc.",...,Telemechanique Electrique,502.0,Telemech_09:51:3b,Not set,Set,Modbus/TCP,1.535061e+09,0.094173,0.094173,mitm
2,3,0.102293,172.27.224.250,172.27.224.70,Modbus/TCP,85,"Response: Trans: 0; Unit: 1, Func: 3: ...",00:80:f4:09:51:3b,Telemech_09:51:3b,Telemechanique Electrique,...,"VMware, Inc.",49499.0,VMware_9d:9e:9e,Not set,Set,Modbus/TCP,1.535061e+09,0.102293,0.008120,mitm
3,4,0.312027,172.27.224.70,172.27.224.250,TCP,60,49499 > 502 [ACK] Seq=13 Ack=32 Win=65206 Len=0,00:0c:29:9d:9e:9e,VMware_9d:9e:9e,"VMware, Inc.",...,Telemechanique Electrique,502.0,Telemech_09:51:3b,Not set,Set,TCP,1.535061e+09,0.312027,0.209734,mitm
4,5,0.406373,172.27.224.70,172.27.224.250,Modbus/TCP,66,"Query: Trans: 0; Unit: 1, Func: 3: ...",00:0c:29:9d:9e:9e,VMware_9d:9e:9e,"VMware, Inc.",...,Telemechanique Electrique,502.0,Telemech_09:51:3b,Not set,Set,Modbus/TCP,1.535061e+09,0.406373,0.094346,mitm


In [34]:
# Check for missing values
missing_values = mitm_df.isnull().sum()
print(missing_values)

No.                      0
Time                     0
SrcIP                    0
DstIP                    0
Protocol                 0
Length                   0
Info                     0
SrcMAC                   0
SrcMACResolved           0
SrcOUIResolved           0
SrcPort             142616
SequenceNumber      155786
SrcOUI                   0
DstMAC                   0
DstMACResolved           0
DstOUI                   0
DstOUResolved       112079
DstPort             142616
DstMACResolved.1         0
SYNFlag             155786
ACKFlag             155786
ProtocolType             0
EpochTime                0
RelativeTime             0
TimeDelta                0
AttackName               0
dtype: int64


In [ ]:
# 1. Handling missing values in SrcPort and DstPort
mitm_df['SrcPort'].fillna(-1, inplace=True)  # Filling missing SrcPort with -1
mitm_df['DstPort'].fillna(-1, inplace=True)  # Filling missing DstPort with -1

# 2. Handling missing values in SequenceNumber
# Assuming TCP packets have SequenceNumber and using median imputation
median_seqnum = mitm_df[mitm_df['Protocol'] == 'TCP']['SequenceNumber'].median()
mitm_df['SequenceNumber'].fillna(median_seqnum, inplace=True)

# 3. Handling missing values in DstOUResolved
mitm_df['DstOUResolved'].fillna('Unknown', inplace=True)  # Filling missing DstOUResolved with 'Unknown'

# 4. Handling missing values in SYNFlag and ACKFlag
mitm_df['SYNFlag'].fillna(False, inplace=True)  # Filling missing SYNFlag with False
mitm_df['ACKFlag'].fillna(False, inplace=True)  # Filling missing ACKFlag with False

In [36]:
# Check for missing values
missing_values = mitm_df.isnull().sum()
print(missing_values)

No.                 0
Time                0
SrcIP               0
DstIP               0
Protocol            0
Length              0
Info                0
SrcMAC              0
SrcMACResolved      0
SrcOUIResolved      0
SrcPort             0
SequenceNumber      0
SrcOUI              0
DstMAC              0
DstMACResolved      0
DstOUI              0
DstOUResolved       0
DstPort             0
DstMACResolved.1    0
SYNFlag             0
ACKFlag             0
ProtocolType        0
EpochTime           0
RelativeTime        0
TimeDelta           0
AttackName          0
dtype: int64


In [37]:
# Convert EpochTime to datetime format and create 'date_time' column
mitm_df['date_time'] = pd.to_datetime(df['EpochTime'], unit='s')

# modbusQuery2Flooding

In [38]:
csv_files = glob.glob('captures1_v2/modbusQuery2Flooding/*.csv')

modbusQuery2Flooding_df = pd.DataFrame()

for csv_file in csv_files:
    try:
        df = pd.read_csv(csv_file)
        modbusQuery2Flooding_df = pd.concat([modbusQuery2Flooding_df, df])
    except Exception as e:
        print(f"Failed to read {csv_file}: {e}")

modbusQuery2Flooding_df['AttackName'] = 'modbusQuery2Flooding'

modbusQuery2Flooding_df.head()

Failed to read captures1_v2/modbusQuery2Flooding\eth2dump-modbusQuery2Flooding30m-1h_1.csv: 'utf-8' codec can't decode byte 0x86 in position 246153: invalid start byte
Failed to read captures1_v2/modbusQuery2Flooding\eth2dump-modbusQuery2Flooding30m-6h_1.csv: Error tokenizing data. C error: Expected 25 fields in line 1258221, saw 38



,No.,Time,SrcIP,DstIP,Protocol,Length,Info,SrcMAC,SrcMACResolved,SrcOUIResolved,...,DstOUResolved,DstPort,DstMACResolved.1,SYNFlag,ACKFlag,ProtocolType,EpochTime,RelativeTime,TimeDelta,AttackName
0,1,0.000000,172.27.224.70,172.27.224.250,TCP,60,49179 > 502 [ACK] Seq=1 Ack=1 Win=64927 Len=0,00:0c:29:9d:9e:9e,VMware_9d:9e:9e,"VMware, Inc.",...,Telemechanique Electrique,502.0,Telemech_09:51:3b,Not set,Set,TCP,1.529551e+09,0.000000,0.000000,modbusQuery2Flooding
1,2,0.057897,172.27.224.251,172.27.224.250,TCP,60,"51111 > 502 [FIN, ACK] Seq=1 Ack=1 Win=2036 ...",48:5b:39:64:40:79,ASUSTekC_64:40:79,ASUSTek COMPUTER INC.,...,Telemechanique Electrique,502.0,Telemech_09:51:3b,Not set,Set,TCP,1.529551e+09,0.057897,0.057897,modbusQuery2Flooding
2,3,0.067569,172.27.224.250,172.27.224.251,TCP,60,502 > 51111 [ACK] Seq=1 Ack=2 Win=8712 Len=0,00:80:f4:09:51:3b,Telemech_09:51:3b,Telemechanique Electrique,...,ASUSTek COMPUTER INC.,51111.0,ASUSTekC_64:40:79,Not set,Set,TCP,1.529551e+09,0.067569,0.009672,modbusQuery2Flooding
3,4,0.068851,172.27.224.250,172.27.224.251,TCP,60,"502 > 51111 [FIN, ACK] Seq=1 Ack=2 Win=8712 ...",00:80:f4:09:51:3b,Telemech_09:51:3b,Telemechanique Electrique,...,ASUSTek COMPUTER INC.,51111.0,ASUSTekC_64:40:79,Not set,Set,TCP,1.529551e+09,0.068851,0.001282,modbusQuery2Flooding
4,5,0.068872,172.27.224.251,172.27.224.250,TCP,60,51111 > 502 [ACK] Seq=2 Ack=2 Win=2036 Len=0,48:5b:39:64:40:79,ASUSTekC_64:40:79,ASUSTek COMPUTER INC.,...,Telemechanique Electrique,502.0,Telemech_09:51:3b,Not set,Set,TCP,1.529551e+09,0.068872,0.000021,modbusQuery2Flooding


In [39]:
# Check for missing values
missing_values = modbusQuery2Flooding_df.isnull().sum()
print(missing_values)

No.                      0
Time                     0
SrcIP                    0
DstIP                    0
Protocol                 0
Length                   0
Info                     0
SrcMAC                   0
SrcMACResolved           0
SrcOUIResolved           0
SrcPort             342203
SequenceNumber      378862
SrcOUI                   0
DstMAC                   0
DstMACResolved           0
DstOUI                   0
DstOUResolved       283669
DstPort             342203
DstMACResolved.1         0
SYNFlag             378862
ACKFlag             378862
ProtocolType             0
EpochTime                0
RelativeTime             0
TimeDelta                0
AttackName               0
dtype: int64


In [ ]:
# 1. Handling missing values in SrcPort and DstPort
modbusQuery2Flooding_df['SrcPort'].fillna(-1, inplace=True)  # Filling missing SrcPort with -1
modbusQuery2Flooding_df['DstPort'].fillna(-1, inplace=True)  # Filling missing DstPort with -1

# 2. Handling missing values in SequenceNumber
# Assuming TCP packets have SequenceNumber and using median imputation
median_seqnum = modbusQuery2Flooding_df[modbusQuery2Flooding_df['Protocol'] == 'TCP']['SequenceNumber'].median()
modbusQuery2Flooding_df['SequenceNumber'].fillna(median_seqnum, inplace=True)

# 3. Handling missing values in DstOUResolved
modbusQuery2Flooding_df['DstOUResolved'].fillna('Unknown', inplace=True)  # Filling missing DstOUResolved with 'Unknown'

# 4. Handling missing values in SYNFlag and ACKFlag
modbusQuery2Flooding_df['SYNFlag'].fillna(False, inplace=True)  # Filling missing SYNFlag with False
modbusQuery2Flooding_df['ACKFlag'].fillna(False, inplace=True)  # Filling missing ACKFlag with False

In [41]:
# Check for missing values
missing_values = modbusQuery2Flooding_df.isnull().sum()
print(missing_values)

No.                 0
Time                0
SrcIP               0
DstIP               0
Protocol            0
Length              0
Info                0
SrcMAC              0
SrcMACResolved      0
SrcOUIResolved      0
SrcPort             0
SequenceNumber      0
SrcOUI              0
DstMAC              0
DstMACResolved      0
DstOUI              0
DstOUResolved       0
DstPort             0
DstMACResolved.1    0
SYNFlag             0
ACKFlag             0
ProtocolType        0
EpochTime           0
RelativeTime        0
TimeDelta           0
AttackName          0
dtype: int64


In [42]:
# Convert EpochTime to datetime format and create 'date_time' column
modbusQuery2Flooding_df['date_time'] = pd.to_datetime(df['EpochTime'], unit='s')

# modbusQueryFlooding

In [43]:
csv_files = glob.glob('captures1_v2/modbusQueryFlooding/*.csv')

modbusQueryFlooding_df = pd.DataFrame()

for csv_file in csv_files:
    try:
        df = pd.read_csv(csv_file)
        modbusQueryFlooding_df = pd.concat([tcpSYNFlood_df, df])
    except Exception as e:
        print(f"Failed to read {csv_file}: {e}")

modbusQueryFlooding_df['AttackName'] = 'modbusQueryFlooding'

modbusQueryFlooding_df.head()

,No.,Time,SrcIP,DstIP,Protocol,Length,Info,SrcMAC,SrcMACResolved,SrcOUIResolved,...,DstOUResolved,DstPort,DstMACResolved.1,SYNFlag,ACKFlag,ProtocolType,EpochTime,RelativeTime,TimeDelta,AttackName
0,1,0.000000,172.27.224.70,172.27.224.250,TCP,60,49179 > 502 [ACK] Seq=1 Ack=1 Win=64865 Len=0,00:0c:29:9d:9e:9e,VMware_9d:9e:9e,"VMware, Inc.",...,Telemechanique Electrique,502.0,Telemech_09:51:3b,Not set,Set,TCP,1.529543e+09,0.000000,0.0,modbusQueryFlooding
1,2,0.111408,172.27.224.70,172.27.224.250,Modbus/TCP,66,"Query: Trans: 0; Unit: 1, Func: 3: ...",00:0c:29:9d:9e:9e,VMware_9d:9e:9e,"VMware, Inc.",...,Telemechanique Electrique,502.0,Telemech_09:51:3b,Not set,Set,Modbus/TCP,1.529543e+09,0.111408,0.111408,modbusQueryFlooding
2,3,0.120144,172.27.224.250,172.27.224.70,Modbus/TCP,85,"Response: Trans: 0; Unit: 1, Func: 3: ...",00:80:f4:09:51:3b,Telemech_09:51:3b,Telemechanique Electrique,...,"VMware, Inc.",49179.0,VMware_9d:9e:9e,Not set,Set,Modbus/TCP,1.529543e+09,0.120144,0.008736,modbusQueryFlooding
3,4,0.238885,HewlettP_8e:40:b3,Spanning-tree-(for-bridges)_00,STP,64,RST. Root = 32768/0/00:18:6e:d7:8a:c0 Cost = ...,d0:7e:28:8e:40:b3,HewlettP_8e:40:b3,Hewlett Packard,...,Unknown,-1.0,Spanning-tree-(for-bridges)_00,False,False,STP,1.529543e+09,0.238885,0.118741,modbusQueryFlooding
4,5,0.327575,172.27.224.70,172.27.224.250,TCP,60,49179 > 502 [ACK] Seq=13 Ack=32 Win=65392 Len=0,00:0c:29:9d:9e:9e,VMware_9d:9e:9e,"VMware, Inc.",...,Telemechanique Electrique,502.0,Telemech_09:51:3b,Not set,Set,TCP,1.529543e+09,0.327575,0.08869,modbusQueryFlooding


In [44]:
# Check for missing values
missing_values = modbusQueryFlooding_df.isnull().sum()
print(missing_values)

No.                     0
Time                    0
SrcIP                   0
DstIP                   0
Protocol                0
Length                  0
Info                  391
SrcMAC                  0
SrcMACResolved          0
SrcOUIResolved          0
SrcPort             26982
SequenceNumber      28831
SrcOUI                  0
DstMAC                  0
DstMACResolved          0
DstOUI                  0
DstOUResolved        9472
DstPort             26982
DstMACResolved.1        0
SYNFlag             28831
ACKFlag             28831
ProtocolType            0
EpochTime               0
RelativeTime            0
TimeDelta           28831
AttackName              0
dtype: int64


In [45]:
# Count the number of packets where TimeDelta should be present but is null
timedelta_missing = modbusQueryFlooding_df[modbusQueryFlooding_df['TimeDelta'].isnull() & ((modbusQueryFlooding_df['Time'].notnull()) & (modbusQueryFlooding_df['EpochTime'].notnull()))].shape[0]
print(f"Number of packets where TimeDelta should be present but is null: {timedelta_missing}")


Number of packets where TimeDelta should be present but is null: 28831


In [46]:
# Sort the DataFrame by EpochTime to ensure the data is in chronological order
modbusQueryFlooding_df.sort_values(by='EpochTime', inplace=True)

# Interpolate missing TimeDelta values based on the EpochTime column
modbusQueryFlooding_df['TimeDelta'] = modbusQueryFlooding_df['EpochTime'].diff().fillna(method='backfill')

# Check if there are any remaining null values in TimeDelta
null_timedelta_count = modbusQueryFlooding_df['TimeDelta'].isnull().sum()
print(f"Number of remaining null values in TimeDelta after interpolation: {null_timedelta_count}")

Number of remaining null values in TimeDelta after interpolation: 0


C:\Users\amani\AppData\Local\Temp\ipykernel_19072\3818620161.py:5: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  modbusQueryFlooding_df['TimeDelta'] = modbusQueryFlooding_df['EpochTime'].diff().fillna(method='backfill')


In [ ]:
# 1. Handling missing values in SrcPort and DstPort
modbusQueryFlooding_df['SrcPort'].fillna(-1, inplace=True)  # Filling missing SrcPort with -1
modbusQueryFlooding_df['DstPort'].fillna(-1, inplace=True)  # Filling missing DstPort with -1

# 2. Handling missing values in SequenceNumber
# Assuming TCP packets have SequenceNumber and using median imputation
median_seqnum = modbusQueryFlooding_df[modbusQueryFlooding_df['Protocol'] == 'TCP']['SequenceNumber'].median()
modbusQueryFlooding_df['SequenceNumber'].fillna(median_seqnum, inplace=True)

# 3. Handling missing values in DstOUResolved
modbusQueryFlooding_df['DstOUResolved'].fillna('Unknown', inplace=True)  # Filling missing DstOUResolved with 'Unknown'

# 4. Handling missing values in SYNFlag and ACKFlag
modbusQueryFlooding_df['SYNFlag'].fillna(False, inplace=True)  # Filling missing SYNFlag with False
modbusQueryFlooding_df['ACKFlag'].fillna(False, inplace=True)  # Filling missing ACKFlag with False

In [48]:
# Check for missing values
missing_values = modbusQueryFlooding_df.isnull().sum()
print(missing_values)

No.                   0
Time                  0
SrcIP                 0
DstIP                 0
Protocol              0
Length                0
Info                391
SrcMAC                0
SrcMACResolved        0
SrcOUIResolved        0
SrcPort               0
SequenceNumber        0
SrcOUI                0
DstMAC                0
DstMACResolved        0
DstOUI                0
DstOUResolved         0
DstPort               0
DstMACResolved.1      0
SYNFlag               0
ACKFlag               0
ProtocolType          0
EpochTime             0
RelativeTime          0
TimeDelta             0
AttackName            0
dtype: int64


In [49]:
# Convert EpochTime to datetime format and create 'date_time' column
modbusQueryFlooding_df['date_time'] = pd.to_datetime(df['EpochTime'], unit='s')

# TcpSYNFlood

In [50]:
csv_files = glob.glob('captures1_v2/tcpSYNFloodDDoS/*.csv')

# Create an empty dataframe to store the combined data
tcpSYNFlood_df = pd.DataFrame()

# Loop through each CSV file and append its contents to the combined dataframe
for csv_file in csv_files:
    try:
        df = pd.read_csv(csv_file)
        tcpSYNFlood_df = pd.concat([tcpSYNFlood_df, df])
    except Exception as e:
        print(f"Failed to read {csv_file}: {e}")

tcpSYNFlood_df['AttackName'] = 'tcpSYNFloodDDoS'

tcpSYNFlood_df.head()

Failed to read captures1_v2/tcpSYNFloodDDoS\eth2dump-tcpSYNFloodDDoS30m-6h_1.csv: 'utf-8' codec can't decode byte 0x86 in position 261916: invalid start byte


,No.,Time,SrcIP,DstIP,Protocol,Length,Info,SrcMAC,SrcMACResolved,SrcOUIResolved,...,DstOUResolved,DstPort,DstMACResolved.1,SYNFlag,ACKFlag,ProtocolType,EpochTime,RelativeTime,TimeDelta,AttackName
0,1,0.000000,172.27.224.70,172.27.224.250,TCP,60,49179 > 502 [ACK] Seq=1 Ack=1 Win=64865 Len=0,00:0c:29:9d:9e:9e,VMware_9d:9e:9e,"VMware, Inc.",...,Telemechanique Electrique,502.0,Telemech_09:51:3b,Not set,Set,TCP,1.529543e+09,0.000000,0.000000,tcpSYNFloodDDoS
1,2,0.111408,172.27.224.70,172.27.224.250,Modbus/TCP,66,"Query: Trans: 0; Unit: 1, Func: 3: ...",00:0c:29:9d:9e:9e,VMware_9d:9e:9e,"VMware, Inc.",...,Telemechanique Electrique,502.0,Telemech_09:51:3b,Not set,Set,Modbus/TCP,1.529543e+09,0.111408,0.111408,tcpSYNFloodDDoS
2,3,0.120144,172.27.224.250,172.27.224.70,Modbus/TCP,85,"Response: Trans: 0; Unit: 1, Func: 3: ...",00:80:f4:09:51:3b,Telemech_09:51:3b,Telemechanique Electrique,...,"VMware, Inc.",49179.0,VMware_9d:9e:9e,Not set,Set,Modbus/TCP,1.529543e+09,0.120144,0.008736,tcpSYNFloodDDoS
3,4,0.238885,HewlettP_8e:40:b3,Spanning-tree-(for-bridges)_00,STP,64,RST. Root = 32768/0/00:18:6e:d7:8a:c0 Cost = ...,d0:7e:28:8e:40:b3,HewlettP_8e:40:b3,Hewlett Packard,...,NaN,NaN,Spanning-tree-(for-bridges)_00,NaN,NaN,STP,1.529543e+09,0.238885,0.118741,tcpSYNFloodDDoS
4,5,0.327575,172.27.224.70,172.27.224.250,TCP,60,49179 > 502 [ACK] Seq=13 Ack=32 Win=65392 Len=0,00:0c:29:9d:9e:9e,VMware_9d:9e:9e,"VMware, Inc.",...,Telemechanique Electrique,502.0,Telemech_09:51:3b,Not set,Set,TCP,1.529543e+09,0.327575,0.088690,tcpSYNFloodDDoS


In [51]:
# Check for missing values
missing_values = tcpSYNFlood_df.isnull().sum()
print(missing_values)

No.                      0
Time                     0
SrcIP                    0
DstIP                    0
Protocol                 0
Length                   0
Info                   391
SrcMAC                   0
SrcMACResolved           0
SrcOUIResolved           0
SrcPort             348148
SequenceNumber      389327
SrcOUI                   0
DstMAC                   0
DstMACResolved           0
DstOUI                   0
DstOUResolved       291851
DstPort             348148
DstMACResolved.1         0
SYNFlag             389327
ACKFlag             389327
ProtocolType             0
EpochTime                0
RelativeTime             0
TimeDelta                0
AttackName               0
dtype: int64


In [52]:
# 1. Handling missing values in SrcPort and DstPort
tcpSYNFlood_df['SrcPort'].fillna(-1, inplace=True)  # Filling missing SrcPort with -1
tcpSYNFlood_df['DstPort'].fillna(-1, inplace=True)  # Filling missing DstPort with -1

# 2. Handling missing values in SequenceNumber
# Assuming TCP packets have SequenceNumber and using median imputation
median_seqnum = tcpSYNFlood_df[tcpSYNFlood_df['Protocol'] == 'TCP']['SequenceNumber'].median()
tcpSYNFlood_df['SequenceNumber'].fillna(median_seqnum, inplace=True)

# 3. Handling missing values in DstOUResolved
tcpSYNFlood_df['DstOUResolved'].fillna('Unknown', inplace=True)  # Filling missing DstOUResolved with 'Unknown'

# 4. Handling missing values in SYNFlag and ACKFlag
tcpSYNFlood_df['SYNFlag'].fillna(False, inplace=True)  # Filling missing SYNFlag with False
tcpSYNFlood_df['ACKFlag'].fillna(False, inplace=True)  # Filling missing ACKFlag with False

# Fill missing Info values with a placeholder like "Unknown"
df['Info'].fillna('Unknown', inplace=True)

C:\Users\amani\AppData\Local\Temp\ipykernel_19072\3233275981.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  tcpSYNFlood_df['SrcPort'].fillna(-1, inplace=True)  # Filling missing SrcPort with -1
C:\Users\amani\AppData\Local\Temp\ipykernel_19072\3233275981.py:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting valu

In [53]:
# Check for missing values
missing_values = tcpSYNFlood_df.isnull().sum()
print(missing_values)

No.                   0
Time                  0
SrcIP                 0
DstIP                 0
Protocol              0
Length                0
Info                391
SrcMAC                0
SrcMACResolved        0
SrcOUIResolved        0
SrcPort               0
SequenceNumber        0
SrcOUI                0
DstMAC                0
DstMACResolved        0
DstOUI                0
DstOUResolved         0
DstPort               0
DstMACResolved.1      0
SYNFlag               0
ACKFlag               0
ProtocolType          0
EpochTime             0
RelativeTime          0
TimeDelta             0
AttackName            0
dtype: int64


In [54]:
# Convert EpochTime to datetime format and create 'date_time' column
tcpSYNFlood_df['date_time'] = pd.to_datetime(df['EpochTime'], unit='s')

# pingFloodDDoS

In [55]:
csv_files = glob.glob('captures1_v2/pingFloodDDoS/*.csv')

pingFloodDDos_df = pd.DataFrame()

for csv_file in csv_files:
    try:
        df = pd.read_csv(csv_file)
        pingFloodDDos_df = pd.concat([pingFloodDDos_df, df])
    except Exception as e:
        print(f"Failed to read {csv_file}: {e}")

pingFloodDDos_df['AttackName'] = 'pingFloodDDoS'

pingFloodDDos_df.head()

,No.,Time,SrcIP,DstIP,Protocol,Length,Info,SrcMAC,SrcMACResolved,SrcOUIResolved,...,DstOUResolved,DstPort,DstMACResolved.1,SYNFlag,ACKFlag,ProtocolType,EpochTime,RelativeTime,TimeDelta,AttackName
0,1,0.000000,172.27.224.70,172.27.224.250,TCP,60,49179 > 502 [ACK] Seq=1 Ack=1 Win=65299 Len=0,00:0c:29:9d:9e:9e,VMware_9d:9e:9e,"VMware, Inc.",...,Telemechanique Electrique,502.0,Telemech_09:51:3b,Not set,Set,TCP,1.529539e+09,0.000000,0.000000,pingFloodDDoS
1,2,0.142182,172.27.224.70,172.27.224.250,Modbus/TCP,66,"Query: Trans: 0; Unit: 1, Func: 3: ...",00:0c:29:9d:9e:9e,VMware_9d:9e:9e,"VMware, Inc.",...,Telemechanique Electrique,502.0,Telemech_09:51:3b,Not set,Set,Modbus/TCP,1.529539e+09,0.142182,0.142182,pingFloodDDoS
2,3,0.145178,172.27.224.250,172.27.224.70,Modbus/TCP,85,"Response: Trans: 0; Unit: 1, Func: 3: ...",00:80:f4:09:51:3b,Telemech_09:51:3b,Telemechanique Electrique,...,"VMware, Inc.",49179.0,VMware_9d:9e:9e,Not set,Set,Modbus/TCP,1.529539e+09,0.145178,0.002996,pingFloodDDoS
3,4,0.358785,172.27.224.70,172.27.224.250,TCP,60,49179 > 502 [ACK] Seq=13 Ack=32 Win=65268 Len=0,00:0c:29:9d:9e:9e,VMware_9d:9e:9e,"VMware, Inc.",...,Telemechanique Electrique,502.0,Telemech_09:51:3b,Not set,Set,TCP,1.529539e+09,0.358785,0.213607,pingFloodDDoS
4,5,0.380756,HewlettP_8e:40:b3,Spanning-tree-(for-bridges)_00,STP,64,RST. Root = 32768/0/00:18:6e:d7:8a:c0 Cost = ...,d0:7e:28:8e:40:b3,HewlettP_8e:40:b3,Hewlett Packard,...,NaN,NaN,Spanning-tree-(for-bridges)_00,NaN,NaN,STP,1.529539e+09,0.380756,0.021971,pingFloodDDoS


In [56]:
# Check for missing values
missing_values = pingFloodDDos_df.isnull().sum()
print(missing_values)

No.                       0
Time                      0
SrcIP                     0
DstIP                     0
Protocol                  0
Length                    0
Info                      0
SrcMAC                    0
SrcMACResolved            0
SrcOUIResolved            0
SrcPort             1803759
SequenceNumber      1845741
SrcOUI                    0
DstMAC                    0
DstMACResolved            0
DstOUI                    0
DstOUResolved        315586
DstPort             1803759
DstMACResolved.1          0
SYNFlag             1845741
ACKFlag             1845741
ProtocolType              0
EpochTime                 0
RelativeTime              0
TimeDelta                 0
AttackName                0
dtype: int64


In [ ]:
# 1. Handling missing values in SrcPort and DstPort
pingFloodDDos_df['SrcPort'].fillna(-1, inplace=True)  # Filling missing SrcPort with -1
pingFloodDDos_df['DstPort'].fillna(-1, inplace=True)  # Filling missing DstPort with -1

# 2. Handling missing values in SequenceNumber
# Assuming TCP packets have SequenceNumber and using median imputation
median_seqnum = pingFloodDDos_df[pingFloodDDos_df['Protocol'] == 'TCP']['SequenceNumber'].median()
pingFloodDDos_df['SequenceNumber'].fillna(median_seqnum, inplace=True)

# 3. Handling missing values in DstOUResolved
pingFloodDDos_df['DstOUResolved'].fillna('Unknown', inplace=True)  # Filling missing DstOUResolved with 'Unknown'

# 4. Handling missing values in SYNFlag and ACKFlag
pingFloodDDos_df['SYNFlag'].fillna(False, inplace=True)  # Filling missing SYNFlag with False
pingFloodDDos_df['ACKFlag'].fillna(False, inplace=True)  # Filling missing ACKFlag with False


In [58]:
# Check for missing values
missing_values = pingFloodDDos_df.isnull().sum()
print(missing_values)

No.                 0
Time                0
SrcIP               0
DstIP               0
Protocol            0
Length              0
Info                0
SrcMAC              0
SrcMACResolved      0
SrcOUIResolved      0
SrcPort             0
SequenceNumber      0
SrcOUI              0
DstMAC              0
DstMACResolved      0
DstOUI              0
DstOUResolved       0
DstPort             0
DstMACResolved.1    0
SYNFlag             0
ACKFlag             0
ProtocolType        0
EpochTime           0
RelativeTime        0
TimeDelta           0
AttackName          0
dtype: int64


In [59]:
# Convert EpochTime to datetime format and create 'date_time' column
pingFloodDDos_df['date_time'] = pd.to_datetime(df['EpochTime'], unit='s')

pingFloodDDos_df.head()

,No.,Time,SrcIP,DstIP,Protocol,Length,Info,SrcMAC,SrcMACResolved,SrcOUIResolved,...,DstPort,DstMACResolved.1,SYNFlag,ACKFlag,ProtocolType,EpochTime,RelativeTime,TimeDelta,AttackName,date_time
0,1,0.000000,172.27.224.70,172.27.224.250,TCP,60,49179 > 502 [ACK] Seq=1 Ack=1 Win=65299 Len=0,00:0c:29:9d:9e:9e,VMware_9d:9e:9e,"VMware, Inc.",...,502.0,Telemech_09:51:3b,Not set,Set,TCP,1.529539e+09,0.000000,0.000000,pingFloodDDoS,2018-05-28 15:38:47.982399940
1,2,0.142182,172.27.224.70,172.27.224.250,Modbus/TCP,66,"Query: Trans: 0; Unit: 1, Func: 3: ...",00:0c:29:9d:9e:9e,VMware_9d:9e:9e,"VMware, Inc.",...,502.0,Telemech_09:51:3b,Not set,Set,Modbus/TCP,1.529539e+09,0.142182,0.142182,pingFloodDDoS,2018-05-28 15:38:48.077550888
2,3,0.145178,172.27.224.250,172.27.224.70,Modbus/TCP,85,"Response: Trans: 0; Unit: 1, Func: 3: ...",00:80:f4:09:51:3b,Telemech_09:51:3b,Telemechanique Electrique,...,49179.0,VMware_9d:9e:9e,Not set,Set,Modbus/TCP,1.529539e+09,0.145178,0.002996,pingFloodDDoS,2018-05-28 15:38:48.084661007
3,4,0.358785,172.27.224.70,172.27.224.250,TCP,60,49179 > 502 [ACK] Seq=13 Ack=32 Win=65268 Len=0,00:0c:29:9d:9e:9e,VMware_9d:9e:9e,"VMware, Inc.",...,502.0,Telemech_09:51:3b,Not set,Set,TCP,1.529539e+09,0.358785,0.213607,pingFloodDDoS,2018-05-28 15:38:48.294389009
4,5,0.380756,HewlettP_8e:40:b3,Spanning-tree-(for-bridges)_00,STP,64,RST. Root = 32768/0/00:18:6e:d7:8a:c0 Cost = ...,d0:7e:28:8e:40:b3,HewlettP_8e:40:b3,Hewlett Packard,...,-1.0,Spanning-tree-(for-bridges)_00,False,False,STP,1.529539e+09,0.380756,0.021971,pingFloodDDoS,2018-05-28 15:38:48.321327925
